In [156]:
import os
import pandas as pd
import numpy as np
import ast
from sklearn.pipeline import Pipeline
from util import def_model, prepare_data, prepare_columntransformer, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
result_path = './results/train_eval/'
data_path = './data/data/'
pretrained_path = './pretrained_models/'

In [157]:
def model_builder(h, lr, dim, dp, cl):
    model = def_model(h, lr, dim, dp)
    ct = prepare_columntransformer(cl)
    model = Pipeline([("preprocess", ct), ("model", model)])
    
    return model
def get_param(path):
    df = pd.read_csv(path_cur, header=None)
    params = ','.join(df.iloc[0].values).split(' >>>')[2]
    params = '{' + params.split('{')[1]
    params = ast.literal_eval(params)
    return params

In [158]:
def evaluate(data_path, pretrained_path, name, params, pretrained = True):
    if name in ['complete', 'complete_dp']:
        data_path = os.path.join(data_path, 'housing_complete' + '.csv')
    else:
        data_path = os.path.join(data_path, name + '.csv')
    X_train, y_train, X_test, y_test = prepare_data(data_path, True)
    
    if pretrained:
        model = pickle.load(open(os.path.join(pretrained_path, name + '.pkl'),'rb'))
    else:
        model = model_builder(params['model__height'],
                          params['model__learning_rate'],
                          X_train.shape[1],
                          params['model__dp'],
                          X_train.columns)

        model.fit(X = X_train, y = y_train, model__epochs = params['model__epochs'],
                  model__batch_size = params['model__batch_size'])
        
    return y_train, model.predict(X_train), y_test, model.predict(X_test)
def evaluate_structure(y, y_hat):
        mape = mean_absolute_percentage_error(y_pred=y_hat, y_true=y)
        mae = mean_absolute_error(y_hat, y)
        mse = mean_squared_error(y_hat, y)
        me = np.mean(y_hat - y)
        sd = np.std(y_hat - y)

        return dict(mae=mae, mape=mape, mse=mse)

In [161]:
# name should be one of: complete, complete_dp, bay, inland, oneHocean, nocean
name = 'complete'
path_cur = os.path.join(result_path, name, name + '.txt')
params = get_param(path_cur)
y_train, y_train_hat, y_test, y_test_hat = evaluate(data_path, pretrained_path, name, params, True)

In [160]:
train_eval = evaluate_structure(y_train, y_train_hat)
test_eval = evaluate_structure(y_test, y_test_hat)